# 数据准备
导入包以及数据

In [1]:
# --- Explore HMM POS Taggers using Brown corpus ---

# In this assignment, you will explore two taggers for a Brown corpus.

# import your packages here
import re
from collections import defaultdict, Counter
import string
import nltk

## 数据加载及探索
加载训练和测试语料库，分别用一个列表表示，列表中的每个元素都是一个句子，这个句子是一个元组列表，其中每一个元组第一个元素是单词，第二个元素是POS标记。

In [2]:
"""
Load all tags and print it out
"""
file_tag = open('NLP_PJ3/brown-tag.txt', mode='r')
tags = [x.strip() for x in file_tag.readlines()]
file_tag.close()
print(tags)

['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


In [3]:
"""
Load corpus files and count sentences and words in datasets
"""
def open_and_count(root):
    f = open(root, mode='r')
    corpus = []
    sent = []
    count_sent = 0
    count_word = 0
    tag_word_dict = defaultdict(int)
    for line in f:
        if re.match('b\d+-\d+', line):
            if sent:
                corpus.append(sent)
                count_sent += 1
            sent = []
        elif line != '\n':
            sent.append(tuple(line.split()))
            count_word += 1
            tag_word_dict[sent[-1][1]] += 1
    corpus.append(sent)
    count_sent += 1
    f.close()
    return corpus, count_sent, count_word, tag_word_dict

In [4]:
train_corpus, train_sent, train_word, train_tag_word = open_and_count('NLP_PJ3/brown-train.txt')
test_corpus, test_sent, test_word, test_tag_word = open_and_count('NLP_PJ3/brown-test.txt')

In [5]:
print('There are {} sentences and {} words in train corpus.'.format(train_sent, train_word))
print('There are {} sentences and {} words in test corpus.'.format(test_sent, test_word))
print('For each tag, the number of words in train and test corpus are as follow:')
for tag in tags:
    print("{}: [{}, {}]".format(tag, train_tag_word[tag], test_tag_word[tag]))

There are 45800 sentences and 928327 words in train corpus.
There are 11540 sentences and 232865 words in test corpus.
For each tag, the number of words in train and test corpus are as follow:
.: [117723, 29842]
ADJ: [66985, 16736]
ADP: [115752, 29014]
ADV: [44765, 11474]
CONJ: [30455, 7696]
DET: [109418, 27601]
NOUN: [220451, 55107]
NUM: [11921, 2953]
PRON: [39657, 9677]
PRT: [23889, 5940]
VERB: [146199, 36551]
X: [1112, 274]


# Baseline Method
使用最大频率标注器进行词性标注，并探索不同的unknown words处理方法对模型效果的影响。

In [6]:
"""
Find the frequency of label for each word in the training data
"""
train_vocab = defaultdict(lambda :defaultdict(int))
for sent in train_corpus:
    for word, tag in sent:
        train_vocab[word][tag] += 1

In [7]:
"""
Predict tag for train datasets. In train corpus there will not be unknown words.
"""
correct = 0
for sent in train_corpus:
    for word, tag in sent:
        pred_tag = max(train_vocab[word], key=lambda x: train_vocab[word][x])
        if pred_tag == tag:
            correct += 1
print('The accuracy of train sample using baseline method is', correct/train_word*100)

The accuracy of train sample using baseline method is 95.71961173164198


In [8]:
"""
Predict tag for test datasets. And try different method to process unknown words.
"""
# Method 1: Simply mark the unknown word with tag "UNK"
correct = 0
for sent in test_corpus:
    for word, tag in sent:
        if word in train_vocab:
            pred_tag = max(train_vocab[word], key=lambda x: train_vocab[word][x])
            if pred_tag == tag:
                correct += 1
        else:
            continue
print('The accuracy of test sample using baseline method is', correct/test_word*100)

The accuracy of test sample using baseline method is 93.03802632426513


在训练集上的准确率超过了90%，也确实高于在测试集上的预测结果。
以上结果是不对unknown words做任何处理的结果，因此之后任何处理unknown words的方法准确率都不应低于该值。

In [9]:
# Method 2: Tag the unknown words with the majority tag among all training samples.
most_freq_tag = max(train_tag_word, key=lambda x:train_tag_word[x])
correct = 0
for sent in test_corpus:
    for word, tag in sent:
        if word in train_vocab:
            pred_tag = max(train_vocab[word], key=lambda x: train_vocab[word][x])
            if pred_tag == tag:
                correct += 1
        else:
            pred_tag = most_freq_tag
            if pred_tag == tag:
                correct += 1
print('The accuracy of test sample using baseline method is', correct/test_word*100)

The accuracy of test sample using baseline method is 94.51871255877869


可以看到测试集上的准确率确实上升了一些，但是无论给这些unknown words标注什么tag，准确率都会有不同幅度的上升。希望找到更加细致的标注划分方式。
下面参考了Adamouization的unknown words标注方法，对unknown words进行进一步的细粒度划分。我将划分之后依旧没有对应tag的words称为real unknown words，请和unknown words区分。

In [10]:
# Method 3: Use word spelling rules tag unknown words
# SUBMETHOD 1: Use 'UNK' tag the real unknown words
NOUN_SUFFIX = ["action", "age", "ance", "cy", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", " ling", "ment", "ness", "or", "ry", "scape", "ship", "dom", "ty"]
VERB_SUFFIX = ["ate", "ify", "ise", "ize", "ed", "ing"]
ADJ_SUFFIX = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
ADV_SUFFIX = ["ward", "wards", "wise"]
UNK_TAG = 'UNK'  # real unknown words tag
PUNC_TAG = '.'
def unknown_words_rules(word):
    if any(char.isdigit() for char in word):
        return 'NUM'
    elif any(char in set(string.punctuation) for char in word):
        return PUNC_TAG
    elif any(word.endswith(suffix) for suffix in NOUN_SUFFIX):
        return 'NOUN'
    elif any(word.endswith(suffix) for suffix in VERB_SUFFIX):
        return 'VERB'
    elif any(word.endswith(suffix) for suffix in ADJ_SUFFIX):
        return 'ADJ'
    elif any(word.endswith(suffix) for suffix in ADV_SUFFIX):
        return 'ADV'
    else:
        return UNK_TAG

In [11]:
correct = 0
for sent in test_corpus:
    for word, tag in sent:
        if word in train_vocab:
            pred_tag = max(train_vocab[word], key=lambda x: train_vocab[word][x])
            if pred_tag == tag:
                correct += 1
        else:
            pred_tag = unknown_words_rules(word)
            if pred_tag == tag:
                correct += 1
print('The accuracy of test sample using baseline method is', correct/test_word*100)

The accuracy of test sample using baseline method is 93.6907650355356


可以看到准确率比最初全部预测为'UNK'要高一些，说明这种划分方法是有效的。接下来，考虑到前面将unknown words预测为tag库中已有的tag可以提高准确率，在这里也将real unknown words标注一个tag。

In [12]:
# SUBMETHOD 2: Tag the real unknown words with the majority tag among all training samples.
UNK_TAG = most_freq_tag
correct = 0
for sent in test_corpus:
    for word, tag in sent:
        if word in train_vocab:
            pred_tag = max(train_vocab[word], key=lambda x: train_vocab[word][x])
            if pred_tag == tag:
                correct += 1
        else:
            pred_tag = unknown_words_rules(word)
            if pred_tag == tag:
                correct += 1
print('The accuracy of test sample using baseline method is', correct/test_word*100)

The accuracy of test sample using baseline method is 94.55822042814506


模型的准确率有稍微的上升。
考虑到也许real unknown words都是比较特别的单词才不在语料库中，试试改变real unknown words的tag为'X'。

In [13]:
# SUBMETHOD3: Tag the real unknown words with 'X'.
UNK_TAG = 'X'
correct = 0
for sent in test_corpus:
    for word, tag in sent:
        if word in train_vocab:
            pred_tag = max(train_vocab[word], key=lambda x: train_vocab[word][x])
            if pred_tag == tag:
                correct += 1
        else:
            pred_tag = unknown_words_rules(word)
            if pred_tag == tag:
                correct += 1
print('The accuracy of test sample using baseline method is', correct/test_word*100)

The accuracy of test sample using baseline method is 93.73628497197947


模型准确率反而有所下降，至少说明real unknown words大部分为'X'这种猜测是有问题的。但是前面的想法有一定道理，使用更加有依据的方法去修正unknown words的tag。
总结一下前面的想法：上面的算法默认将识别不出来的单词标记为所有训练数据集中出现频率最高的那个标签，但这其中可能有一个问题是：**所有训练语料库**中出现最频繁的标签有可能并不等价于**unknown words语料库**真实标签出现最频繁的标签（比如可能就是因为某个tag出现的频率太低才会导致unknown words的出现）。
尝试用训练集中只出现了一次的单词模拟测试集中的unknown words语料库。我在这里将训练集中只出现一次的单词构成的语料库成为稀疏语料库，代码中使用unique words来表示对应单词。

In [14]:
# Count the most frequent tag between the words that only appear once in train datasets
word_freq_dist = Counter([word[0] for sent in train_corpus for word in sent])
unique_word_tag_dist = defaultdict(int)
for word in word_freq_dist.keys():
    if word_freq_dist[word] != 1:
        continue
    unique_word_tag_dist[list(train_vocab[word].keys())[0]] += 1
print('The most frequent tag between unique words in train datasets is', max(unique_word_tag_dist, key=lambda x:unique_word_tag_dist[x]))

The most frequent tag between unique words in train datasets is NOUN


确实证实了NOUN是出现频率最高的tag，无论是在完整数据集还是稀疏数据集中。
同样的idea，第二个可能存在的问题是：上面使用的unknown_word_rules在完整数据集中看很合理，但在稀疏数据集中是否也保持一致呢？进一步地，在稀疏数据集中的most frequent tag又是否会和real unknown word中的most frequent tag一致呢？
和上面类似，采用在稀疏数据集上模拟的方法来验证。

In [15]:
tag2tag = defaultdict(lambda :defaultdict(int))
for word in word_freq_dist.keys():
    if word_freq_dist[word] != 1:
        continue
    if any(char.isdigit() for char in word):
        tag2tag['NUM'][list(train_vocab[word].keys())[0]] += 1
    elif any(char in set(string.punctuation) for char in word):
        tag2tag['.'][list(train_vocab[word].keys())[0]] += 1
    elif any(word.endswith(suffix) for suffix in NOUN_SUFFIX):
        for suffix in NOUN_SUFFIX:
            if word.endswith(suffix):
                tag2tag[suffix][list(train_vocab[word].keys())[0]] += 1
                break
    elif any(word.endswith(suffix) for suffix in VERB_SUFFIX):
        for suffix in VERB_SUFFIX:
            if word.endswith(suffix):
                tag2tag[suffix][list(train_vocab[word].keys())[0]] += 1
                break
    elif any(word.endswith(suffix) for suffix in ADJ_SUFFIX):
        for suffix in ADJ_SUFFIX:
            if word.endswith(suffix):
                tag2tag[suffix][list(train_vocab[word].keys())[0]] += 1
                break
    elif any(word.endswith(suffix) for suffix in ADV_SUFFIX):
        for suffix in ADV_SUFFIX:
            if word.endswith(suffix):
                tag2tag[suffix][list(train_vocab[word].keys())[0]] += 1
                break
    else:
        tag2tag['UNK'][list(train_vocab[word].keys())[0]] += 1   # 对real unknown word进一步调整

来检查是否有区别：

In [16]:
print(max(tag2tag['NUM'], key=lambda x:tag2tag['NUM'][x]))
print(max(tag2tag['.'], key=lambda x:tag2tag['.'][x]))
print(max(tag2tag['UNK'], key=lambda x:tag2tag['UNK'][x]))

NUM
NOUN
NOUN


可以看到'NUM'的tag没有变化，real unknown word的most frequent tag也确实是'NOUN'没有变。但是原本被判定为'.'的tag变成了'NOUN'！
更进一步，看看suffix的原定tag是否有出现变化的：

In [17]:
print('The tag change in NOUN_SUFFIX:')
for suffix in NOUN_SUFFIX:
    if tag2tag[suffix] and max(tag2tag[suffix], key=lambda x:tag2tag[suffix][x]) != 'NOUN':
        print('The tag of suffix {} change from NOUN to {}.'.format(suffix, max(tag2tag[suffix], key=lambda x:tag2tag[suffix][x])))
print('************************************************')
print('The tag change in VERB_SUFFIX:')
for suffix in VERB_SUFFIX:
    if tag2tag[suffix] and max(tag2tag[suffix], key=lambda x:tag2tag[suffix][x]) != 'VERB':
        print('The tag of suffix {} change from VERB to {}.'.format(suffix, max(tag2tag[suffix], key=lambda x:tag2tag[suffix][x])))
print('************************************************')
print('The tag change in ADJ_SUFFIX:')
for suffix in ADJ_SUFFIX:
    if tag2tag[suffix] and max(tag2tag[suffix], key=lambda x:tag2tag[suffix][x]) != 'ADJ':
        print('The tag of suffix {} change from ADJ to {}.'.format(suffix, max(tag2tag[suffix], key=lambda x:tag2tag[suffix][x])))
print('************************************************')
print('The tag change in NOUN_SUFFIX:')
for suffix in ADV_SUFFIX:
    if tag2tag[suffix] and max(tag2tag[suffix], key=lambda x:tag2tag[suffix][x]) != 'ADV':
        print('The tag of suffix {} change from ADV to {}.'.format(suffix, max(tag2tag[suffix], key=lambda x:tag2tag[suffix][x])))
print('************************************************')

The tag change in NOUN_SUFFIX:
************************************************
The tag change in VERB_SUFFIX:
The tag of suffix ise change from VERB to NOUN.
************************************************
The tag change in ADJ_SUFFIX:
The tag of suffix ese change from ADJ to NOUN.
The tag of suffix i change from ADJ to NOUN.
The tag of suffix ly change from ADJ to ADV.
************************************************
The tag change in NOUN_SUFFIX:
************************************************


可以看到有四种suffix的tag都发生了变化！在对应的suffix tag列表进行修改，同时也修改前面探索过的UNK-TAG、PUNC-TAG。

In [18]:
VERB_SUFFIX.remove('ise')
ADJ_SUFFIX.remove('ese')
ADJ_SUFFIX.remove('i')
ADJ_SUFFIX.remove('ly')
NOUN_SUFFIX.append('ise')
NOUN_SUFFIX.append('ese')
NOUN_SUFFIX.append('i')
ADV_SUFFIX.append('ly')
UNK_TAG = 'NOUN'
PUNC_TAG = 'NOUN'

In [19]:
correct = 0
for sent in test_corpus:
    for word, tag in sent:
        if word in train_vocab:
            pred_tag = max(train_vocab[word], key=lambda x: train_vocab[word][x])
            if pred_tag == tag:
                correct += 1
        else:
            pred_tag = unknown_words_rules(word)
            if pred_tag == tag:
                correct += 1
print('The accuracy of test sample using baseline method is', correct/test_word*100)

The accuracy of test sample using baseline method is 94.92667425332274


可以看到效果有一定的提升，目前的模型是以上所有方法里面最好的！

## 总结
在训练集上：

|Method|Accuracy|
|---|---|
|Baseline tagger|95.71961173164198|

在测试集上：

|Unknown Words Method|Accuracy|
|---|---|
|naive 'UNK'|93.03802632426513|
|most frequent tag|94.51871255877869|
|suffix tagging + naive 'UNK'|93.6907650355356|
|suffix tagging + most frequent tag|94.55822042814506|
|suffix tagging + other tag 'X'|93.73628497197947|
|**corrected suffix tagging**|94.92667425332274|

# Task 3. Build an HMM tagger.
根据上面对unknown words的处理方法对比，可以看到最后一种效果是最好的，因为它的分类是最细的，考虑情况是最完善的。因此，在HMM的模型中，我也考虑使用这种unknown words的处理方法。为了能让模型处理这些unknown words，首先要对训练集只出现一次的单词进行特征模糊，这是为了构建稀疏数据集，然后要对测试集只出现一次的单词同样进行特征模糊，这是为了让模型能够识别它。

In [21]:
# Process the train dataset
remove_words = []
for i in range(train_sent):
    for j in range(len(train_corpus[i])):
        word = train_corpus[i][j][0]
        tag = train_corpus[i][j][1]
        if word_freq_dist[word] != 1:
            continue
        train_corpus[i][j] = ('UNK-' + unknown_words_rules(word), tag)
        remove_words.append(word)
# Process the test dataset
for i in range(test_sent):
    for j in range(len(test_corpus[i])):
        word = test_corpus[i][j][0]
        tag = test_corpus[i][j][1]
        if word not in train_vocab or word in remove_words:
            test_corpus[i][j] = ('UNK-' + unknown_words_rules(word), tag)

In [22]:
vocab = set([word[0] for sent in train_corpus for word in sent])

In [23]:
def get_trans_matrix(corpus):
    trans = nltk.ConditionalFreqDist([(sent[word_index][1], sent[word_index + 1][1]) for sent in corpus for word_index in range(len(sent)-1)])
    trans = nltk.ConditionalProbDist(trans, nltk.MLEProbDist)
    return trans

def get_emission_matrix(corpus):
    emission = nltk.ConditionalFreqDist([(word[1], word[0]) for sent in corpus for word in sent])
    emission = nltk.ConditionalProbDist(emission, nltk.MLEProbDist)
    return emission

def get_prior_matrix(corpus):
    # prior = nltk.UniformProbDist([sent[0][1] for sent in corpus])
    prior = nltk.MLEProbDist(nltk.FreqDist([sent[0][1] for sent in corpus]))
    return prior

In [24]:
train_trans = get_trans_matrix(train_corpus)
train_emission = get_emission_matrix(train_corpus)
train_prior = get_prior_matrix(train_corpus)
model = nltk.tag.HiddenMarkovModelTagger(symbols = vocab, states = set(tags), transitions = train_trans, outputs = train_emission, priors = train_prior)

In [25]:
# Test the model separately on train corpus and test corpus
# On train corpus
correct = 0
for sent in train_corpus:
    seq = [word[0] for word in sent]
    true_label = [word[1] for word in sent]
    pred_label = model.tag(seq)
    correct += sum([true_label[i] == pred_label[i][1] for i in range(len(true_label))])
print('The accuracy of model HMM on train corpus is', correct/train_word*100)

The accuracy of model HMM on train corpus is 96.89204342866253


In [26]:
correct = 0
for sent in test_corpus:
    seq = [word[0] for word in sent]
    true_label = [word[1] for word in sent]
    pred_label = model.tag(seq)
    correct += sum([true_label[i] == pred_label[i][1] for i in range(len(true_label))])
print('The accuracy of model HMM on test corpus is', correct/test_word*100)

The accuracy of model HMM on test corpus is 96.31546174822321


可以看到HMM在训练集和测试集上的表现均有一定提升。这是因为HMM考虑到了语义的前后联系，利用到了tag之间变换的更多信息，因此建模更加贴合实际，也能减轻一些words本身歧义带来的干扰。
另外，可以看到这种标注方法在训练集和测试集上的表现差距非常小（比baseline方法小得多），说明HMM标注器模型的鲁棒性非常好。